## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df=application_df.drop(['EIN','NAME','SPECIAL_CONSIDERATIONS','STATUS'], axis=1)
application_df.head(5)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [3]:
# Transformation of the ASK_AMT column to its log
import numpy as np
application_df['ASK_AMT (log)']=np.log10(application_df['ASK_AMT'])

In [4]:
# Drop the ASK_AMT values from the dataframe
application_df.drop(["ASK_AMT"],axis=1)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL,ASK_AMT (log)
0,T10,Independent,C1000,ProductDev,Association,0,1,3.698970
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1,5.035790
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0,3.698970
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1,3.825556
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1,5.154089
...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,0,3.698970
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,0,3.698970
34296,T3,CompanySponsored,C2000,Preservation,Association,0,0,3.698970
34297,T5,Independent,C3000,ProductDev,Association,0,1,3.698970


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff=200
application_types_to_replace=[]
for _ in application_df['APPLICATION_TYPE'].unique():
    result=application_df[application_df['APPLICATION_TYPE']==_].count()
    if result[0]<cutoff:
        application_types_to_replace.append(_)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff2=1800
classifications_to_replace=[]
for _ in application_df['CLASSIFICATION'].unique():
    result2=application_df[application_df['CLASSIFICATION']==_].count()
    if result2[0]<cutoff2:
        classifications_to_replace.append(_)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [7]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies=pd.get_dummies(application_df, dtype=int)

In [8]:
# Split our preprocessed data into our features and target arrays

y = application_df["IS_SUCCESSFUL"].values
X = application_dummies.drop(["IS_SUCCESSFUL"],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
# Create a MinMaxScaler instance
scaler = MinMaxScaler()

# Fit the MinMaxScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
X_train_scaled.min()

0.0

In [11]:
X_train_scaled.max()

1.0

In [12]:
len(X_train[0])

41

## Compile, Train and Evaluate the Model

In [13]:
# Was a missing package in Colab.
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 1.4 MB/s eta 0:00:00


Optimization Test with a more complex Tuner

In [14]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model3(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=41))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=hp.Choice('activation',['relu','sigmoid'])))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [15]:
# Import the kerastuner library
import keras_tuner as kt

tuner3 = kt.Hyperband(
    create_model3,
    objective="val_accuracy",
    max_epochs=100,
    directory="project_5",
    hyperband_iterations=2)

In [16]:
# Run the kerastuner search for best hyperparameters
tuner3.search(X_train_scaled,y_train,epochs=40,validation_data=(X_test_scaled,y_test))

Trial 508 Complete [00h 05m 26s]
val_accuracy: 0.7308454513549805

Best val_accuracy So Far: 0.7343440055847168
Total elapsed time: 05h 12m 43s


In [17]:
# Get the optimal hyperparameters
best_hps= tuner3.get_best_hyperparameters(1)[0]

# get the best model
best_model = tuner3.get_best_models(1)[0]

In [18]:
print(best_hps.values)

{'activation': 'sigmoid', 'first_units': 11, 'num_layers': 4, 'units_0': 21, 'units_1': 26, 'units_2': 16, 'units_3': 21, 'units_4': 26, 'tuner/epochs': 100, 'tuner/initial_epoch': 34, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0482'}


In [19]:
# Evaluate best model against full test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5727 - accuracy: 0.7343 - 632ms/epoch - 2ms/step
Loss: 0.5727332830429077, Accuracy: 0.7343440055847168


In [20]:
best_model.save('AlphabetSoupCharity_Optimization5.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
